In [28]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
pd.set_option('display.max_colwidth', 500)

In [29]:
df_bigtech = pd.read_csv('bigtech_lobby_issue_2011-2021.csv')

In [30]:
df = df_bigtech[['season', 'year', 'amazon_specific_issue', 
       'facebook_specific_issue','google_specific_issue',
        'apple_specific_issue', 'microsoft_specific_issue']]

In [31]:
df.head(1)

,season,year,amazon_specific_issue,facebook_specific_issue,google_specific_issue,apple_specific_issue,microsoft_specific_issue
0,2021 2nd Quarter - Report,2021,"[' Issues related to broadband, satellite communications, spectrum, device accessibility, Section 230 reform, content moderation, and online video. ', ' Issues related to intellectual property, including copyright reform and online infringement, counterfeits, including the SHOP Safe Act of 2021 (H.R. 3429/S. 1843), music licensing, patent reform, Copyright Office modernization, issues related to the Digital Millennium Copyright Act, and issues related to intermediary liabili...","[' S. 936, Integrity, Notification, and Fairness in Online Retail Marketplaces for Consumers Act (INFORM Consumers Act); Discussions regarding general intellectual property issues, including copyright reform ', ' Discussions regarding cyber security, data security, encryption, platform integrity, election integrity, content policy, and terrorism; Issues related to voter suppression/interference, political ads and misinformation policies; Issues related to Spanish langua...","[' Regulation of online advertising Privacy and competition issues in online advertising ', ' Intellectual property enforcement Patent standards and quality; venue; post-grant review Digital Millennium Copyright Act US Copyright Office modernization Register of Copyrights Intermediary liability for copyright infringing content generated by users Music licensing ', ' Privacy and data security issues Transparency related to the Foreign Intelligence Surveillance...","[' General patent policy Issues related to standard essential patents and fair, reasonable, and non-discriminatory (FRAND) licensing ', ' Issues related to foreign and domestic tax ', "" Issues related to student privacy Issues related to computer science education Providing general information about Apple's inclusion and diversity programs related to education Providing general information about Apple's coding and education programs "", ' Issue...","["" Licensing, competition, trade, and government procurement in the software industry. Health IT procurement; health reform and related IT issues. Private sector preparedness; cyber research and development. Standards and interoperability. Competitiveness and high-tech sector. Policy issues in cloud computing. Open government, government IT modernization, open data, and technology choice. Supply chain security efforts within US government. Cybersecurity policies, legislation, protecting cri..."


## Let's vectorize!
https://investigate.ai/text-analysis/a-simple-explanation-of-tf-idf/

In [32]:
import re
import os
from collections import Counter
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
import Stemmer

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
%matplotlib inline

In [33]:
"""
MEMO
remove numbers = .replace('\d+', 'NUM', regex=True)
remove numbers with 4 digits = .replace('\d{4}', '', regex=True)  ex:2020, 2018
"""

"\nMEMO\nremove numbers = .replace('\\d+', 'NUM', regex=True)\nremove numbers with 4 digits = .replace('\\d{4}', '', regex=True)  ex:2020, 2018\n"

In [34]:
## Stemmer : copy&paste from classwork10. However, it does NOT seem work accurately. I decided NOT to use this time.

# stemmer = Stemmer.Stemmer('en')
# analyzer = TfidfVectorizer().build_analyzer()

# class StemmedTfidfVectorizer(TfidfVectorizer):
#     def build_analyzer(self):
#         analyzer = super(TfidfVectorizer, self).build_analyzer()
#         return lambda doc: stemmer.stemWords(analyzer(doc))

# token_pattern='(?u)\\b\\w+\\b'

## Amazon

In [35]:
texts = df.amazon_specific_issue.replace('\d{4}', '', regex=True)

my_stop_words = text.ENGLISH_STOP_WORDS.union(['fy','year','fiscal', 'law','act','issues','related','including','discussions','regarding'])


# Make a vectorizer
# token_pattern='(?u)\\b\\w+\\b' because we want to match one-character words (a, I, etc)
vectorizer = TfidfVectorizer(ngram_range=(2,2), 
                             min_df=0.1, max_df=0.7, max_features=100, use_idf=True,
                            stop_words=my_stop_words, norm='l1')
# Learn and count the words in df.content
matrix = vectorizer.fit_transform(texts)
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names(),
                        index=df.season)

# words_df.head()

In [36]:
test = words_df.T.sort_values('2021 2nd Quarter - Report', ascending=False)

amazon100 = pd.Series(test.index)

In [37]:
test.to_csv('freq_words_amazon.csv', index=True)

## Facebook

In [38]:
texts = df.facebook_specific_issue.replace('\d{4}', '', regex=True)

my_stop_words = text.ENGLISH_STOP_WORDS.union(['fy','year','fiscal', 'law','act','issues','related','including','discussions','regarding'])


# Make a vectorizer
# token_pattern='(?u)\\b\\w+\\b' because we want to match one-character words (a, I, etc)
vectorizer = TfidfVectorizer(ngram_range=(2,2), 
                             min_df=0.1, max_df=0.7, max_features=100, use_idf=True,
                            stop_words=my_stop_words, norm='l1')
# Learn and count the words in df.content
matrix = vectorizer.fit_transform(texts)
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names(),
                        index=df.season)

# words_df.head()

In [39]:
test = words_df.T.sort_values('2021 2nd Quarter - Report', ascending=False)

facebook100 = pd.Series(test.index)

In [40]:
test.to_csv('freq_words_facebook.csv', index=True)

## Google

In [41]:
texts = df.google_specific_issue.replace('\d{4}', '', regex=True)

my_stop_words = text.ENGLISH_STOP_WORDS.union(['fy','year','fiscal', 'law','act','issues','related','including','discussions','regarding'])


# Make a vectorizer
# token_pattern='(?u)\\b\\w+\\b' because we want to match one-character words (a, I, etc)
vectorizer = TfidfVectorizer(ngram_range=(2,2), 
                             min_df=0.1, max_df=0.7, max_features=100, use_idf=True,
                            stop_words=my_stop_words, norm='l1')
# Learn and count the words in df.content
matrix = vectorizer.fit_transform(texts)
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names(),
                        index=df.season)

# words_df.head()

In [42]:
test = words_df.T.sort_values('2021 2nd Quarter - Report', ascending=False)

google100 = pd.Series(test.index)

In [43]:
test.to_csv('freq_words_google.csv', index=True)

## Apple

In [44]:
texts = df.apple_specific_issue.replace('\d{4}', '', regex=True)

my_stop_words = text.ENGLISH_STOP_WORDS.union(['fy','year','fiscal', 'law','act','issues','related','including','discussions','regarding'])


# Make a vectorizer
# token_pattern='(?u)\\b\\w+\\b' because we want to match one-character words (a, I, etc)
vectorizer = TfidfVectorizer(ngram_range=(2,2), 
                             min_df=0.1, max_df=0.7, max_features=100, use_idf=True,
                            stop_words=my_stop_words, norm='l1')
# Learn and count the words in df.content
matrix = vectorizer.fit_transform(texts)
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names(),
                        index=df.season)

# words_df.head()

In [45]:
test = words_df.T.sort_values('2021 2nd Quarter - Report', ascending=False)

apple100 = pd.Series(test.index)

In [46]:
test.to_csv('freq_words_apple.csv', index=True)

## Microsoft

In [47]:
texts = df.microsoft_specific_issue.replace('\d{4}', '', regex=True)

my_stop_words = text.ENGLISH_STOP_WORDS.union(['fy','year','fiscal', 'law','act','issues','related','including','discussions','regarding','12'])


# Make a vectorizer
# token_pattern='(?u)\\b\\w+\\b' because we want to match one-character words (a, I, etc)
vectorizer = TfidfVectorizer(ngram_range=(2,2), 
                             min_df=0.1, max_df=0.7, max_features=100, use_idf=True,
                            stop_words=my_stop_words, norm='l1')
# Learn and count the words in df.content
matrix = vectorizer.fit_transform(texts)
words_df = pd.DataFrame(matrix.toarray(),
                        columns=vectorizer.get_feature_names(),
                        index=df.season)

# words_df.head()

In [48]:
test = words_df.T.sort_values('2021 2nd Quarter - Report', ascending=False)

microsoft100 = pd.Series(test.index)

In [49]:
test.to_csv('freq_words_microsoft.csv', index=True)

In [50]:
bigtech100 = pd.concat([amazon100, facebook100, google100, apple100, microsoft100], axis = 1)

In [51]:
bigtech100 = bigtech100.rename(columns={0: 'amazon', 1:'facebook', 2: 'google', 3: 'apple', 4: 'microsoft'})

In [52]:
bigtech100.to_csv('bigtech_100words.csv', index=False)

In [53]:
#DF based on 2021Q2 tfIdf
bigtech100

,amazon,facebook,google,apple,microsoft
0,public 116,personal data,supply chain,data security,expanding access
1,116 136,election integrity,intermediary liability,technologies immigration,computer science
2,cares public,journalism competition,abroad mutual,deferred action,access computer
3,appropriations public,competition preservation,content generated,action childhood,science education
4,authorization public,platform integrity,safe harbor,arrivals program,facial recognition
5,consolidated appropriations,honest ads,liability copyright,childhood arrivals,digital trade
6,implementation cares,education science,innovation daca,climate change,high skilled
7,covid 19,science technology,generated users,development autonomous,copyright office
8,public 115,content platform,efforts internet,autonomous vehicle,office modernization
9,government procurement,consideration deferred,copyright infringing,vehicle technologies,youth youth
